# Credit card fraud detector

In [7]:
import sys
sys.path.insert(0, './src/')

## Investigate and process the data

In [8]:
import boto3
from package import config

instance_type = 'ml.m5.large'
s3 = boto3.resource('s3', region_name=config.AWS_REGION)
object = s3.Object(f"{config.SOLUTIONS_S3_BUCKET}-{config.AWS_REGION}",f"{config.SOLUTION_NAME}/data/creditcardfraud.zip")
object.download_file("creditcardfraud.zip")

ModuleNotFoundError: No module named 'boto3'

In [ ]:
import numpy as np 
import pandas as pd

data = pd.read_csv('creditcard_2023.csv', delimiter=',')

In [ ]:
print(data.columns)
data[['Time', 'V1', 'V2', 'V27', 'V28', 'Amount', 'Class']].describe()

In [ ]:
nonfrauds, frauds = data.groupby('Class').size()
print('Number of frauds: ', frauds)
print('Number of non-frauds: ', nonfrauds)
print('Percentage of fradulent data:', 100.*frauds/(frauds + nonfrauds))

In [ ]:
feature_columns = data.columns[:-1]
label_column = data.columns[-1]

features = data[feature_columns].values.astype('float32')
labels = (data[label_column].values).astype('float32')

## Training

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    features, labels, test_size=0.1, random_state=42)

## Unsupervised Learning

In [ ]:
import os
import sagemaker
from package import config

session = sagemaker.Session()
bucket = config.MODEL_DATA_S3_BUCKET
prefix = 'fraud-classifier'

In [ ]:
from sagemaker import RandomCutForest

# specify general training job information
rcf = RandomCutForest(role=config.SAGEMAKER_IAM_ROLE,
                      train_instance_count=1,
                      train_instance_type=instance_type,
                      data_location='s3://{}/{}/'.format(bucket, prefix),
                      output_path='s3://{}/{}/output'.format(bucket, prefix),
                      base_job_name="{}-rcf".format(config.SOLUTION_PREFIX),
                      num_samples_per_tree=512,
                      num_trees=50)

In [ ]:
rcf.fit(rcf.record_set(X_train))

### Host Random Cut Forest

In [ ]:
rcf_predictor = rcf.deploy(
    model_name="{}-rcf".format(config.SOLUTION_PREFIX),
    endpoint_name="{}-rcf".format(config.SOLUTION_PREFIX),
    initial_instance_count=1,
    instance_type=instance_type)

In [ ]:
from sagemaker.predictor import csv_serializer, json_deserializer

rcf_predictor.content_type = 'text/csv'
rcf_predictor.serializer = csv_serializer
rcf_predictor.accept = 'application/json'
rcf_predictor.deserializer = json_deserializer

### Test Random Cut Forest

In [ ]:
def predict_rcf(current_predictor, data, rows=500):
    split_array = np.array_split(data, int(data.shape[0] / float(rows) + 1))
    predictions = []
    for array in split_array:
        array_preds = [s['score'] for s in current_predictor.predict(array)['scores']]
        predictions.append(array_preds)

    return np.concatenate([np.array(batch) for batch in predictions])

In [ ]:
positives = X_test[y_test == 1]
positives_scores = predict_rcf(rcf_predictor, positives)

negatives = X_test[y_test == 0]
negatives_scores = predict_rcf(rcf_predictor, negatives)

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
sns.set(color_codes=True)

In [ ]:
sns.distplot(positives_scores, label='fraud', bins=20)
sns.distplot(negatives_scores, label='not-fraud', bins=20)
plt.legend()

## Supervised Learning

### Prepare Data and Upload to S3

In [ ]:
import io
import sklearn
from sklearn.datasets import dump_svmlight_file   

buf = io.BytesIO()

sklearn.datasets.dump_svmlight_file(X_train, y_train, buf)
buf.seek(0);

In [ ]:
key = 'fraud-dataset'
subdir = 'base'
boto3.resource('s3', region_name=config.AWS_REGION).Bucket(bucket).Object(os.path.join(prefix, 'train', subdir, key)).upload_fileobj(buf)

s3_train_data = 's3://{}/{}/train/{}/{}'.format(bucket, prefix, subdir, key)
print('Uploaded training data location: {}'.format(s3_train_data))

output_location = 's3://{}/{}/output'.format(bucket, prefix)
print('Training artifacts will be uploaded to: {}'.format(output_location))

In [ ]:
from sagemaker.amazon.amazon_estimator import get_image_uri

container = get_image_uri(boto3.Session().region_name, 'xgboost', repo_version='0.90-2')

In [ ]:
from math import sqrt

# Because the data set is so highly skewed, we set the scale position weight conservatively,
# as sqrt(num_nonfraud/num_fraud).
# Other recommendations for the scale_pos_weight are setting it to (num_nonfraud/num_fraud).
scale_pos_weight = sqrt(np.count_nonzero(y_train==0)/np.count_nonzero(y_train))
hyperparams = {
    "max_depth":5,
    "subsample":0.8,
    "num_round":100,
    "eta":0.2,
    "gamma":4,
    "min_child_weight":6,
    "silent":0,
    "objective":'binary:logistic',
    "eval_metric":'auc',
    "scale_pos_weight": scale_pos_weight
}

In [ ]:
clf = sagemaker.estimator.Estimator(container,
   role=config.SAGEMAKER_IAM_ROLE,
   hyperparameters=hyperparams,
    train_instance_count=1, 
   train_instance_type=instance_type,
    output_path=output_location,
    sagemaker_session=session,
   base_job_name="{}-xgb".format(config.SOLUTION_PREFIX))

In [ ]:
clf.fit({'train': s3_train_data})

### Host Classifier

In [ ]:
from sagemaker.predictor import csv_serializer

predictor = clf.deploy(initial_instance_count=1,
                       model_name="{}-xgb".format(config.SOLUTION_PREFIX),
                       endpoint_name="{}-xgb".format(config.SOLUTION_PREFIX),
                       instance_type=instance_type,
                       serializer=csv_serializer,
                       deserializer=None,
                       content_type='text/csv')

## Evaluation

In [ ]:
# Because we have a large test set, we call predict on smaller batches
def predict(current_predictor, data, rows=500):
    split_array = np.array_split(data, int(data.shape[0] / float(rows) + 1))
    predictions = ''
    for array in split_array:
        predictions = ','.join([predictions, current_predictor.predict(array).decode('utf-8')])

    return np.fromstring(predictions[1:], sep=',')

In [ ]:
raw_preds = predict(predictor, X_test)

In [ ]:
from sklearn.metrics import balanced_accuracy_score, cohen_kappa_score

# scikit-learn expects 0/1 predictions, so we threshold our raw predictions
y_preds = np.where(raw_preds > 0.5, 1, 0)
print("Balanced accuracy = {}".format(balanced_accuracy_score(y_test, y_preds)))
print("Cohen's Kappa = {}".format(cohen_kappa_score(y_test, y_preds)))

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix

def plot_confusion_matrix(y_true, y_predicted):

    cm  = confusion_matrix(y_true, y_predicted)
    # Get the per-class normalized value for each cell
    cm_norm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
    
    # We color each cell according to its normalized value, annotate with exact counts.
    ax = sns.heatmap(cm_norm, annot=cm, fmt="d")
    ax.set(xticklabels=["non-fraud", "fraud"], yticklabels=["non-fraud", "fraud"])
    ax.set_ylim([0,2])
    plt.title('Confusion Matrix')
    plt.ylabel('Real Classes')
    plt.xlabel('Predicted Classes')
    plt.show()

In [ ]:
plot_confusion_matrix(y_test, y_preds)

In [ ]:
from sklearn.metrics import classification_report

print(classification_report(
    y_test, y_preds, target_names=['non-fraud', 'fraud']))

### Keep sending test traffic to the endpoint via lambda

In [ ]:
force_traffic_generation = False
#force_traffic_generation = True # If in Studio, set to True after you've updated you permissions

In [ ]:
from threading import Thread
from package.generate_endpoint_traffic import generate_traffic

if config.SAGEMAKER_MODE == "NotebookInstance" or force_traffic_generation:
    thread = Thread(target = generate_traffic, args=[np.copy(X_test)])
    thread.start()


In [ ]:
from IPython.display import Markdown as md
if config.SAGEMAKER_MODE == "NotebookInstance" or force_traffic_generation:
    md(f"[Link to Lambda Monitoring](https://{config.AWS_REGION}.console.aws.amazon.com/lambda/home?region={config.AWS_REGION}#/functions/{config.SOLUTION_PREFIX}-event-processor?tab=monitoring)")

### SMOTE

In [ ]:
from imblearn.over_sampling import SMOTE

smote = SMOTE(random_state=42)
X_smote, y_smote = smote.fit_resample(X_train, y_train)

In [ ]:
from collections import Counter
print(sorted(Counter(y_smote).items()))

In [ ]:
smote_buf = io.BytesIO()

# Dump the SMOTE data into a buffer
sklearn.datasets.dump_svmlight_file(X_smote, y_smote, smote_buf)
smote_buf.seek(0);

# Upload from the buffer to S3
key = 'fraud-dataset-smote'
subdir = 'smote'
boto3.resource('s3', region_name=config.AWS_REGION).Bucket(bucket).Object(os.path.join(prefix, 'train', subdir, key)).upload_fileobj(smote_buf)

s3_smote_train_data = 's3://{}/{}/train/{}/{}'.format(bucket, prefix, subdir, key)
print('Uploaded training data location: {}'.format(s3_smote_train_data))

smote_output_location = 's3://{}/{}/smote-output'.format(bucket, prefix)
print('Training artifacts will be uploaded to: {}'.format(smote_output_location))

In [ ]:
# No need to scale weights after SMOTE resampling, so we remove that parameter
hyperparams.pop("scale_pos_weight", None)
smote_xgb = sagemaker.estimator.Estimator(container,
                                        role=config.SAGEMAKER_IAM_ROLE,
                                        hyperparameters=hyperparams,
                                        train_instance_count=1, 
                                        train_instance_type=instance_type,
                                        output_path=smote_output_location,
                                        sagemaker_session=session,
                                        base_job_name="{}-xgb-smote".format(config.SOLUTION_PREFIX))

We are now ready to fit the model, which should take around 5 minutes to complete.

In [ ]:
smote_xgb.fit({'train': s3_smote_train_data})

After fitting the model we can check its performance to compare it against the base XGBoost model. The deployment will take around 10 minutes.

In [ ]:
smote_predictor = smote_xgb.deploy(initial_instance_count=1,
                       model_name="{}-xgb-smote".format(config.SOLUTION_PREFIX),
                       endpoint_name="{}-xgb-smote".format(config.SOLUTION_PREFIX),
                       instance_type=instance_type)

# Specify input and output formats.
smote_predictor.content_type = 'text/csv'
smote_predictor.serializer = csv_serializer
smote_predictor.deserializer = None

In [ ]:
smote_raw_preds = predict(smote_predictor, X_test)
smote_preds = np.where(smote_raw_preds > 0.5, 1, 0)

In [ ]:
print("Balanced accuracy = {}".format(balanced_accuracy_score(y_test, smote_preds)))
print("Cohen's Kappa = {}".format(cohen_kappa_score(y_test, smote_preds)))

In [ ]:
plot_confusion_matrix(y_test, smote_preds)

In [ ]:
print(classification_report(
    y_test, smote_preds, target_names=['non-fraud', 'fraud']))

Due to the randomness of XGBoost your results may vary, but overall, you should see a large increase in non-fraud cases being classified as fraud (false positives). The reason this happens is because SMOTE has oversampled the fraud class so much that it's increased its overlap in feature space with the non-fraud cases.
Since Cohen's Kappa gives more weight to false positives than balanced accuracy does, the metric drops significantly, as does the precision and F1 score for fraud cases. However, we can bring a balance between the metrics again by adjusting our classification threshold.

So far we've been using 0.5 as the threshold between labeling a point as fraud or not. We can try different thresholds to see if they affect the result of the classification. To evaluate we'll use the balanced accuracy and Cohen's Kappa metrics.

In [ ]:
for thres in np.linspace(0.1, 0.9, num=9):
    smote_thres_preds = np.where(smote_raw_preds > thres, 1, 0)
    print("Threshold: {:.1f}".format(thres))
    print("Balanced accuracy = {:.3f}".format(balanced_accuracy_score(y_test, smote_thres_preds)))
    print("Cohen's Kappa = {:.3f}\n".format(cohen_kappa_score(y_test, smote_thres_preds)))

We see that Cohen's Kappa keeps increasing along with the threshold, without a significant loss in balanced accuracy. This adds a useful knob to our model: We can keep a low threshold if we care more about not missing any fraudulent cases, or we can increase the threshold to try to minimize the number of false positives.

## Clean up

We will leave the unsupervised and base XGBoost endpoints running at the end of this notebook so we can handle incoming event streams using the Lambda function. The solution will automatically clean up the endpoints when deleted, however, don't forget to ensure the prediction endpoints are deleted when you're done. You can do that at the Amazon SageMaker console in the Endpoints page. Or you can run `predictor_name.delete_endpoint()` here.

In [ ]:
# Uncomment to clean up endpoints
rcf_predictor.delete_model()
rcf_predictor.delete_endpoint()
predictor.delete_model()
predictor.delete_endpoint()
smote_predictor.delete_model()
smote_predictor.delete_endpoint()
sm_client = boto3.client('sagemaker', region_name=config.AWS_REGION)
waiter = sm_client.get_waiter('endpoint_deleted')
waiter.wait(EndpointName="{}-xgb-smote".format(config.SOLUTION_PREFIX))
waiter.wait(EndpointName="{}-xgb".format(config.SOLUTION_PREFIX))
waiter.wait(EndpointName="{}-rcf".format(config.SOLUTION_PREFIX))
